<a href="https://colab.research.google.com/github/amien1410/amien-scrapers/blob/main/Cnn_Stock_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://edition.cnn.com/markets/stocks/VNET"
res = requests.get(url)
res

<Response [200]>

In [ ]:
soup = BeautifulSoup(res.text)
soup.title

<title>VNET Stock Quote Price and Forecast | CNN</title>

In [ ]:
soup

In [ ]:
# price momentum 200-days average
# https://production.dataviz.cnn.io/insights/share_price/LCID

# PE, company name, market cap, current price, 52 week range, pe ratio competitor highest and lowest
# https://production.dataviz.cnn.io/quote/competitors/LCID/5

# Tiprank
# https://widgets.tipranks.com/content/v2/cnn/smartscoresmall/index.html?ticker=VNET

# https://www.tipranks.com/api/assets?tickers=VNET
# https://market.tipranks.com/api/quotes/GetQuotes?app_name=tr&tickers=CMG



In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

def calculate_price_change(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    today = datetime.today().date()

    # Define time ranges
    dates = {
        "1 Month": today - timedelta(days=30),
        "6 Months": today - timedelta(days=182),  # ~6 months
        "1 Year": today - timedelta(days=365)
    }

    current_price = stock.history(period="1d")["Close"].iloc[-1]
    print(f"\nCurrent Price of {ticker_symbol.upper()}: ${current_price:.2f}")

    for label, date in dates.items():
        past_data = stock.history(start=date - timedelta(days=5), end=date + timedelta(days=5))
        if not past_data.empty:
            past_price = past_data["Close"].iloc[0]
            change_percent = ((current_price - past_price) / past_price) * 100
            print(f"{label} Change: {change_percent:.2f}% (Past Price: ${past_price:.2f})")
        else:
            print(f"{label} Change: No data available around {date}")

# Example usage
calculate_price_change("CMG")  # Replace with any stock symbol


Current Price of CMG: $53.93
1 Month Change: 2.65% (Past Price: $52.54)
6 Months Change: -4.45% (Past Price: $56.44)
1 Year Change: -4.73% (Past Price: $56.61)


In [ ]:
def get_change_percent(current_price, past_price):
  return f"Change: {((current_price - past_price) / past_price) * 100}"

In [ ]:
get_change_percent(53.93, 53.56)

'Change: 0.6908140403285986'

In [3]:
tipranksHeaders = {
  "accept":  "application/json, text/plain, */*",
  "accept-encoding":  "gzip, deflate, br, zstd",
  "accept-language":  "en-US,en;q=0.9,id;q=0.8,fa;q=0.7,ar;q=0.6,ms;q=0.5,ja;q=0.4,es;q=0.3",
  "cookie":  "personal-message=none; tr-plan-id=0; tr-plan-name=free; tr-experiments-version=1.14; tipranks-experiments=%7b%22Experiments%22%3a%5b%7b%22Name%22%3a%22general_A%22%2c%22Variant%22%3a%22v2%22%2c%22SendAnalytics%22%3afalse%7d%2c%7b%22Name%22%3a%22general_B%22%2c%22Variant%22%3a%22v1%22%2c%22SendAnalytics%22%3afalse%7d%2c%7b%22Name%22%3a%22general_C%22%2c%22Variant%22%3a%22v7%22%2c%22SendAnalytics%22%3afalse%7d%5d%7d; tipranks-experiments-slim=general_A%3av2%7cgeneral_B%3av1%7cgeneral_C%3av7; rbzid=LtANLSeCHMfmYXZm2cZ1L3pKEQQyDsPMsrJfGhmykxwDJgkIngd2w2QYM+9ntvEzEZ+v9JVUyaIDG641/Jf6wmrB61Y4+Wy1/v1n93ZQXn+Ohn4GkizVvYP9yUPk+yTjZHP6J7uvesn+SJFgtJVlokN8lS6D5anbjGICm6zPyv0A0/wSDH8eDYCvIVj5I018ciIges02MyaX0Y6akaGoPd3FTt8E/fE10hkhYXmRdxDqTxp71HMpU6TxR877TVBSu137k77ox/jhfr302vKQ0w__",
  "priority":  "u=1, i",
  "referer":  "https://www.tipranks.com/stocks/tsla/forecast",
  "sec-ch-ua":  '"Not)A;Brand";v="8", "Chromium";v="138", "Microsoft Edge";v="138"',
  "sec-ch-ua-mobile":  "?0",
  "sec-ch-ua-platform":  "Windows",
  "sec-fetch-dest":  "empty",
  "sec-fetch-mode":  "cors",
  "sec-fetch-site":  "same-origin",
  "user-agent":  "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0"
}

res = requests.get("https://www.tipranks.com/stocks/tsla/stock-forecast/payload.json", headers=tipranksHeaders)
res

<Response [200]>

In [ ]:
res.json()['models']['stocks'][5]['analystRatings']['all']

{'id': 'neutral',
 'buy': 13,
 'sell': 8,
 'hold': 13,
 'total': 34,
 'priceTarget': {'value': 298.93},
 'highPriceTarget': 500,
 'lowPriceTarget': 19.05}

In [4]:
res = requests.get("https://widgets.tipranks.com/api/widgets/stockAnalysisOverview/?tickers=vnet")
res.json()

[{'ticker': 'vnet',
  'hedgeFundTrendValue': 2513468.0,
  'smartScore': 9,
  'bloggerSectorAvg': 0.6783,
  'bloggerBullishSentiment': 1.0,
  'insidersLast3MonthsSum': 0.0,
  'newsSentimentsBearishPercent': 0.5,
  'newsSentimentsBullishPercent': 0.5,
  'investorHoldingChangeLast7Days': 0.0787219929026804,
  'investorHoldingChangeLast30Days': 0.0596711821480425,
  'priceTarget': 14.03,
  'convertedPriceTarget': 14.03,
  'convertedPriceTargetCurrencyId': 1,
  'fundamentalsReturnOnEquity': 0.022766,
  'fundamentalsAssetGrowth': 0.374915,
  'technicalsTwelveMonthsMomentum': 3.0334,
  'sma': 'Positive',
  'analystConsensus': 'StrongBuy',
  'hedgeFundTrend': 'Increased',
  'insiderTrend': '-',
  'investorSentiment': 'VeryPositive',
  'newsSentiment': 'Neutral',
  'bloggerConsensus': 'Bullish',
  'marketCountryId': 1,
  'isomic': 'XNAS'}]